In [4]:
import pandas as pd
import numpy as np
import string as s
import seaborn as sns
import math
import tables
import matplotlib
import matplotlib.pyplot as plt
from itertools import groupby
import scipy.stats as stats
from scipy.ndimage.interpolation import shift
# matplotlib.style.use('ggplot')
# %matplotlib inline
matplotlib.rcParams['figure.figsize'] = (12, 6)
# matplotlib.rcParams['figure.facecolor'] = 'white'


targeted sequencing QC

In [5]:
wkdir = '/home/szong/projects/QC/targeted_seq/BLGSP/'

In [10]:
f1 = '{0}{1}'.format(wkdir, 'NCI_capture_of_BLGSP_samples_level_2_QC_results_9_runs.xls')
df1 = pd.read_excel(f1, sheetname='result').dropna()
df1.head(2)

,pool library,library,sample_id,index,# of on-target reads,# of off-target reads,% of reads on target (from total reads),% of reads off target (from total reads),fold coverage (incl. duplicate reads)
0,IX6039.CC47LANXX_1.1stLane,A78672,BLGSP-71-19-00106,CCAGCA,5358375.0,3674778.0,0.593190,0.406810,138.0
1,IX6039.CC47LANXX_1.1stLane,A78675,BLGSP-71-19-00114,TACAAG,8695613.0,5051912.0,0.632522,0.367478,225.0


In [13]:
df1.columns

Index(['pool library', 'library', 'sample_id', 'index', '# of on-target reads',
       '# of off-target reads', '% of reads on target (from total reads)',
       '% of reads off target (from total reads)',
       'fold coverage (incl. duplicate reads)'],
      dtype='object')

In [17]:
df_cov = df1.groupby('library').agg({'fold coverage (incl. duplicate reads)':sum,\
                            '% of reads on target (from total reads)':'mean',\
                            '% of reads off target (from total reads)': 'mean'})

In [29]:
df_cov.head(2)

,fold coverage (incl. duplicate reads),% of reads on target (from total reads),% of reads off target (from total reads)
library,,,
A78670,724.0,0.617571,0.382429
A78671,490.0,0.620799,0.379201


In [18]:
f2 = '{0}{1}'.format(wkdir, 'SeqVal121.RawAlignedBaseCount.20180306.xlsx')
df2 = pd.read_excel(f2, sheetname='raw.out').dropna()
df2.head(2)

,Pool,Library,Run ID,Raw Reads,Aligned Reads,Raw Bases Sequenced,Number of bases Aligned
0,IX6039,A78670,run220967,29139547,29117860,3642443375,3639732500
1,IX6039,A78670,run221274,29040603,29017215,3630075375,3627151875


In [20]:
df2.columns

Index(['Pool', 'Library', 'Run ID', 'Raw Reads', 'Aligned Reads',
       'Raw Bases Sequenced', 'Number of bases Aligned'],
      dtype='object')

In [26]:
df3 = df2.groupby('Library').agg({'Raw Bases Sequenced':sum, 'Number of bases Aligned':sum})
df3.head(2)

,Raw Bases Sequenced,Number of bases Aligned
Library,,
A78670,10889102000,10880776000
A78671,7309201250,7304921375


In [28]:
df_bases = df3/1000000000

In [32]:
dfm1 = pd.merge(df_cov, df_bases, left_index=True, right_index=True)
dfm1.head(2)

,fold coverage (incl. duplicate reads),% of reads on target (from total reads),% of reads off target (from total reads),Raw Bases Sequenced,Number of bases Aligned
library,,,,,
A78670,724.0,0.617571,0.382429,10.889102,10.880776
A78671,490.0,0.620799,0.379201,7.309201,7.304921


In [34]:
f4 = '{0}{1}'.format(wkdir, 'probe_cov_files.txt')
df4 =  pd.read_csv(f4, header=None)
df4.columns = ['file']
df4.head(2)

,file
0,A78670.dOutput_parsed.tsv
1,A78671.dOutput_parsed.tsv


In [44]:
def get_ocov_bases(df4):
    lst = []
    for idx, f in df4.iterrows():
        file = f['file']
        lib = file.split('.')[0]
        df = pd.read_csv('{0}{1}'.format(wkdir, file), sep='\t')
        bases_w_ocov = df.loc[0,'Number of Bases With This Coverage (Or More) (Not Genomically Unique)']
        lst.append([lib, bases_w_ocov])
    dff = pd.DataFrame(lst)
    dff.columns = ['library', 'bases_w_ocoverage']
    dff = dff.set_index('library', drop=True)
    return dff

In [45]:
dff = get_ocov_bases(df4)

In [47]:
dff.head(2)

,bases_w_ocoverage
library,
A78670,13358
A78671,18635


In [48]:
dfm2 = pd.merge(dfm1, dff, left_index=True, right_index=True)

In [51]:
dfm2 = dfm2.sort_index()

In [53]:
of = '{0}{1}'.format(wkdir, 'qc.txt')
dfm2.to_csv(of, sep='\t')